In [ ]:
try:
    from google.colab import drive
    from google.colab import output
    drive.mount('/content/drive')
    output.enable_custom_widget_manager()
    !jupyter nbextension enable --py widgetsnbextension
except:
    print("Not running in Google Colab, skipping drive mount and widget manager setup.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
import os
import pickle
import random
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Layer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow_probability as tfp
from tqdm import tqdm
import warnings
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import (Input, Dense, LayerNormalization,
                                   Dropout, GlobalAveragePooling1D, Conv1D,
                                   Embedding, Multiply, Concatenate, Lambda)



print("All dependencies successfully imported!")


tfd = tfp.distributions
# Configuration
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
sns.set_style("whitegrid")
pd.set_option('display.max_columns', 50)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Constants
SHIFT = 10800
FREQUENCY_START_MHZ = 0
FREQUENCY_END_MHZ = 10934 - SHIFT
BGS_RAW_TXT_PATH = '/content/drive/MyDrive/OstadSharif/FINAL/data/BGS_raw.txt'
BPS_RAW_TXT_PATH = '/content/drive/MyDrive/OstadSharif/FINAL/data/BPS_raw.txt'

MODELS_DIR = '/content/drive/MyDrive/OstadSharif/FINAL/models'
LOG_DIR = '/content/drive/MyDrive/OstadSharif/FINAL/logs'
RESULTS_DIR = '/content/drive/MyDrive/OstadSharif/FINAL/results/PDNN_V3'

All dependencies successfully imported!


In [ ]:
def txt_to_numpy_array(file_path: str) -> np.ndarray:
    print(f"Loading data from {file_path}...")
    with open(file_path, 'r') as f:
        lines = f.readlines()
    data = np.array([list(map(float, line.strip().split(','))) for line in lines])
    data = data[:, ~np.isnan(data).any(axis=0)]  # Remove columns with NaNs
    print(f"Loaded data shape: {data.shape}")
    return data

In [ ]:
real_bgs = txt_to_numpy_array(BGS_RAW_TXT_PATH)
real_bps = txt_to_numpy_array(BPS_RAW_TXT_PATH)[:,1:]
actual_num_freq_points, actual_num_distance_points = real_bgs.shape
distance_axis_idx = np.arange(actual_num_distance_points)
frequency_axis_mhz = np.linspace(FREQUENCY_START_MHZ, FREQUENCY_END_MHZ, actual_num_freq_points)


wvec = frequency_axis_mhz
xvec = distance_axis_idx
x0 = [40, 15]

Loading data from /content/drive/MyDrive/OstadSharif/FINAL/data/BGS_raw.txt...
Loaded data shape: (68, 4500)
Loading data from /content/drive/MyDrive/OstadSharif/FINAL/data/BPS_raw.txt...
Loaded data shape: (68, 4501)


In [ ]:
def bgs_scale(yvec, scaling='Standard'):
    if scaling == 'Standard':
        scaler = StandardScaler()
    elif scaling == 'MinMax':
        scaler = MinMaxScaler()
    elif scaling == 'Robust':
        scaler = RobustScaler()
    else:
        raise ValueError("Invalid scaling type")
    return scaler.fit_transform(yvec.reshape(-1, 1)).flatten()

In [ ]:
def bps_scale(yvec, scaling='Standard'):
    if scaling is 'Standard':
        scaler = StandardScaler()
    elif scaling is 'MinMax':
        scaler = MinMaxScaler(feature_range=(-0.5,0.5))
    elif scaling is 'Robust':
        scaler = RobustScaler()
    else:
        raise ValueError()
    return scaler.fit_transform(yvec.reshape(len(yvec),1)).flatten()

In [ ]:
def lorentzian_abs(x, g0, w, x0):
    return g0 * (w**2 / (4 * (x - x0)**2 + w**2))

In [ ]:
def bgs_data_gen(wvec,bgs_parms,noise=0.0):
    n_w=len(wvec)
    bfs,fwhm = bgs_parms
    amp=1.0
    func=amp/(1+((wvec-bfs)/(fwhm/2))**2)
    func=func+noise*np.random.normal(0,1,n_w)
    return func

In [ ]:
def bps_data_gen(wvec,bps_parms,noise=0.0):
    n_w=len(wvec)
    bfs, fwhm = bps_parms
    amp=1.0
    func=amp*(wvec-bfs)*(fwhm/2)/((wvec-bfs)**2+(fwhm/2)**2)
    func=func+noise*np.random.normal(0,1,n_w)
    return func


In [ ]:
def ls_fit(data,x0,mode='bgs'):
    xdata=data[:,0]
    ydataraw=data[:,1]
    if mode == 'bgs':
        ydata=bgs_scale(ydataraw,'MinMax')
        def fun(xdata,fpeak,fwhm):
            return bgs_data_gen(xdata,[fpeak,fwhm])
    elif mode == 'bps':
        ydata=bps_scale(ydataraw,'MinMax')
        def fun(xdata,fpeak,fwhm):
            return bps_data_gen(xdata,[fpeak,fwhm])
    else:
        raise ValueError()
    x,cov_x=curve_fit(fun, xdata,ydata, p0=x0,bounds=(np.array([0,5]), np.array([80,60])))
    x_err=[np.sqrt(cov_x[j,j]) for j in range(x.size)]
    return x,x_err

In [ ]:
def batch_ls_fit(raw_data, wvec, x0, mode='bgs'):
    n_x = raw_data.shape[1]
    parmat = np.zeros((n_x, 2))
    parerr = np.zeros((n_x, 2))
    for i in tqdm(range(n_x)):
        data = np.column_stack([wvec, raw_data[:, i]])
        parmat[i, :], parerr[i, :] = ls_fit(data, x0, mode)
    return parmat, parerr


In [ ]:
wvec = frequency_axis_mhz

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Dropout, LayerNormalization, Input, Conv1D, Embedding
from tensorflow.keras.layers import GlobalAveragePooling1D, Flatten
from tensorflow.keras.models import Model

# --------------------------
# Probabilistic Output Layer
# --------------------------
@tf.keras.utils.register_keras_serializable()
class ProbabilisticOutputLayer(Layer):
    def __init__(self, n_out=2, **kwargs):
        super().__init__(**kwargs)
        self.n_out = n_out

    def call(self, inputs):
        if self.n_out == 2:
            loc = inputs[..., :2]
            scale = 1e-5 + tf.nn.softplus(0.1 * inputs[..., 2:])
            return tf.concat([loc, scale], axis=-1)
        elif self.n_out == 1:
            loc = inputs[..., :1]
            scale = 1e-5 + tf.nn.softplus(0.1 * inputs[..., 1:])
            return tf.concat([loc, scale], axis=-1)
        else:
            raise ValueError("Invalid number of outputs")

    def get_config(self):
        config = super().get_config()
        config.update({'n_out': self.n_out})
        return config


# --------------------------
# Transformer Block
# --------------------------
@tf.keras.utils.register_keras_serializable()
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


# --------------------------
# Frequency-Aware Embedding
# --------------------------
@tf.keras.utils.register_keras_serializable()
class FrequencyAwareEmbedding(Layer):
    def __init__(self, max_freq, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.max_freq = max_freq
        self.embed_dim = embed_dim
        self.freq_embedding = Embedding(input_dim=max_freq+1, output_dim=embed_dim)

    def call(self, inputs):
        freq_indices = tf.cast(inputs[:, :, 0], tf.int32)
        amplitude_values = inputs[:, :, 1]
        freq_embeddings = self.freq_embedding(freq_indices)
        amplitude_values = tf.expand_dims(amplitude_values, axis=-1)
        return freq_embeddings * amplitude_values

    def get_config(self):
        config = super().get_config()
        config.update({
            'max_freq': self.max_freq,
            'embed_dim': self.embed_dim
        })
        return config


# --------------------------
# Add Noise Layer
# --------------------------
@tf.keras.utils.register_keras_serializable()
class AddNoise(Layer):
    def __init__(self, stddev=0.01, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, inputs, training=False):
        if training:
            noise = tf.random.normal(tf.shape(inputs), mean=0.0, stddev=self.stddev)
            return inputs + noise
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({'stddev': self.stddev})
        return config


# --------------------------
# Attention Pooling Layer
# --------------------------
@tf.keras.utils.register_keras_serializable()
class AttentionPooling(Layer):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.dense = Dense(units, activation='tanh')
        self.context_vector = Dense(1, use_bias=False)

    def call(self, inputs):
        score = self.context_vector(self.dense(inputs))   # (batch, seq_len, 1)
        weights = tf.nn.softmax(score, axis=1)
        context = tf.reduce_sum(weights * inputs, axis=1)
        return context


In [ ]:
class LRTracker(keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self._lrs = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current_lr = keras.backend.get_value(self.model.optimizer.learning_rate)
        self._lrs.append(current_lr)
        logs['lr'] = current_lr

@tf.keras.utils.register_keras_serializable()
def negloglik(y_true, y_pred):
    if y_pred.shape[-1] == 4:  # n_out == 2
        loc = y_pred[..., :2]  # BFS and FWHM means
        scale = y_pred[..., 2:]  # BFS and FWHM stds
        distribution = tfd.Independent(
            tfd.Normal(loc=loc, scale=scale),
            reinterpreted_batch_ndims=1
        )
    elif y_pred.shape[-1] == 2:  # n_out == 1
        loc = y_pred[..., :1]
        scale = y_pred[..., 1:]
        distribution = tfd.Normal(loc=loc, scale=scale)
    else:
        raise ValueError("Invalid output shape")
    return -distribution.log_prob(y_true)

In [ ]:
class PDNN:
    def __init__(self, wvec=None, channels=2, scaler='Standard', mode='bgs', batch_size=1000, max_epochs=500):
        self.mode = mode
        self.model = None
        self.model_history = 'Network has not been trained'
        self.wvec = wvec
        self.input_shape = (len(wvec), channels)

        if self.mode == 'bgs':
          self.n_out = 2

        elif self.mode == 'bps':
          self.n_out = 1
        else:
          print("error mode")
        self.scaling = scaler
        self.batch_size = batch_size
        self.max_epochs = max_epochs
        self.model_path = os.path.join(MODELS_DIR, f'{self.mode}_pdnn_v3.keras')
        self.log_dir = LOG_DIR
        self.results_dir = RESULTS_DIR
        self.lr_tracker = LRTracker()

    def data_batchgen(self, parmat):
        n_w, n_c = self.input_shape
        N = parmat.shape[0]
        func_parms = parmat[:, 0:self.n_out]
        func_results = np.zeros((N, n_w, n_c))
        for i in tqdm(range(N), "generating data..."):
            func_results[i, :, 0] = self.wvec
            if self.mode == 'bgs':
                bgsvec = bgs_data_gen(self.wvec, parmat[i, :2], noise=parmat[i, 2])
                func_results[i, :, 1] = bgs_scale(bgsvec, scaling=self.scaling)
            elif self.mode == 'bps':
                bpsvec = bps_data_gen(self.wvec, parmat[i, :2], noise=parmat[i, 2])
                func_results[i, :, 1] = bps_scale(bpsvec, scaling=self.scaling)
            else:
                raise ValueError("Invalid mode")
        return func_results, func_parms

    def create_model(self, max_freq=134):
        inputs = Input(shape=self.input_shape)

        # Frequency-aware embedding
        x = FrequencyAwareEmbedding(max_freq, 128)(inputs)
        x = AddNoise(stddev=0.01)(x)

        # Local feature extraction with CNN
        x = Conv1D(128, kernel_size=7, padding='same', activation='relu')(x)
        x = Conv1D(128, kernel_size=5, padding='same', activation='relu')(x)

        # Stacked Transformers
        x = TransformerBlock(embed_dim=128, num_heads=4, ff_dim=256)(x)
        x = TransformerBlock(embed_dim=128, num_heads=8, ff_dim=512)(x)
        x = TransformerBlock(embed_dim=128, num_heads=8, ff_dim=512)(x)

        # Attention pooling
        x = AttentionPooling(units=128)(x)

        # Dense head
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.3)(x)
        x = Dense(2 * self.n_out, activation='linear')(x)

        outputs = ProbabilisticOutputLayer(n_out=self.n_out)(x)

        self.model = Model(inputs=inputs, outputs=outputs, name="SpectralModel_V3")


    def compile_model(self, initial_lr=0.005):
        print("Compiling model with negative log-likelihood loss...")
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=initial_lr),
            loss=negloglik
        )
        self.callbacks = [
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.01,
                patience=32,
                verbose=1,
                restore_best_weights=True,
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=8,
                verbose=0,
                mode='auto',
                min_delta=1e-3,
                min_lr=1e-6
            ),
            ModelCheckpoint(
                self.model_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=1
            ),
            self.lr_tracker
        ]

    def train_and_visualize(self, X, y, epochs=None, batch_size=None):
        print("Starting model training...")
        epochs = epochs or self.max_epochs
        batch_size = batch_size or self.batch_size
        self.history = self.model.fit(
            X, y,
            validation_split=0.3,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=self.callbacks,
            verbose=2,
            shuffle=True
        )
        plt.figure(figsize=(18, 12))
        sns.set_style("whitegrid")
        palette = sns.color_palette("husl", 8)
        plt.subplot(2, 2, 1)
        plt.plot(self.history.history['loss'], label='Train', color=palette[0])
        plt.plot(self.history.history['val_loss'], label='Validation', color=palette[1])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend()
        plt.subplot(2, 2, 4)
        if hasattr(self.lr_tracker, '_lrs'):
            plt.plot(self.lr_tracker._lrs, label='Learning Rate', color=palette[6])
            plt.title('Learning Rate Schedule')
            plt.ylabel('LR')
            plt.xlabel('Epoch')
            plt.yscale('log')
            plt.legend()
        else:
            plt.text(0.5, 0.5, 'Learning Rate Not Tracked', ha='center', va='center')
            plt.title('Learning Rate Schedule (Not Available)')
            plt.axis('off')
        plt.tight_layout()
        plt.show()
        print("Training completed.")

    def full_pipeline(self, N=300000, minvals=(10.0, 10.0, 0.0), maxvals=(130.0, 60.0, 0.05)):
        print("Starting full pipeline...")
        print("Generating synthetic data...")
        parmat = np.random.uniform(minvals, maxvals, (N, 3))
        X, y = self.data_batchgen(parmat)
        self.create_model()
        self.compile_model()
        print("Fitting the model...")
        self.train_and_visualize(X, y, epochs=512, batch_size=1024)
        print("Pipeline completed.")

    def batch_predict(self, data):
        n_w, n_c = self.input_shape
        N = data.shape[1]
        inputs = np.zeros((N, n_w, n_c))
        for i in range(N):
            inputs[i, :, 0] = self.wvec
            if self.mode == 'bgs':
                inputs[i, :, 1] = bgs_scale(data[:, i], scaling=self.scaling)
            elif self.mode == 'bps':
                inputs[i, :, 1] = bps_scale(data[:, i], scaling=self.scaling)
            else:
                raise ValueError("Invalid mode")

        predictions = self.model.predict(inputs)
        if self.n_out == 2:
            par_mean = predictions[:, :2]  # BFS mean, FWHM mean
            par_std = predictions[:, 2:]   # BFS std, FWHM std
        else:
            par_mean = predictions[:, :1]  # BFS mean
            par_std = predictions[:, 1:]   # BFS std
        return par_mean, par_std

    def file_dnn_predict(self, file):
        raw_data = txt_to_numpy_array(file)
        par_mean, par_std = self.batch_predict(raw_data)
        return par_mean, par_std

    def load_model(self):
        if self.model is None:
            self.model = tf.keras.models.load_model(
                self.model_path,
                custom_objects={
                    'TransformerBlock': TransformerBlock,
                    'AttentionPooling': AttentionPooling,
                    'ProbabilisticOutputLayer': ProbabilisticOutputLayer,
                    'negloglik': negloglik
                }
            )
        return self.model

In [ ]:
pdnn = PDNN(wvec=frequency_axis_mhz, mode='bps')
# pdnn.load_model()
pdnn.full_pipeline()

Starting full pipeline...
Generating synthetic data...


generating data...: 100%|██████████| 300000/300000 [02:39<00:00, 1876.94it/s]


Compiling model with negative log-likelihood loss...
Fitting the model...
Starting model training...
Epoch 1/512

Epoch 1: val_loss improved from inf to 5.65170, saving model to /content/drive/MyDrive/OstadSharif/FINAL/models/bps_pdnn_v3.keras
206/206 - 168s - 817ms/step - loss: 75.7349 - val_loss: 5.6517 - learning_rate: 0.0050 - lr: 0.0050
Epoch 2/512

Epoch 2: val_loss improved from 5.65170 to 4.98740, saving model to /content/drive/MyDrive/OstadSharif/FINAL/models/bps_pdnn_v3.keras
206/206 - 170s - 825ms/step - loss: 5.4460 - val_loss: 4.9874 - learning_rate: 0.0050 - lr: 0.0050
Epoch 3/512

Epoch 3: val_loss improved from 4.98740 to 4.97128, saving model to /content/drive/MyDrive/OstadSharif/FINAL/models/bps_pdnn_v3.keras
206/206 - 142s - 689ms/step - loss: 5.0558 - val_loss: 4.9713 - learning_rate: 0.0050 - lr: 0.0050
Epoch 4/512

Epoch 4: val_loss did not improve from 4.97128
206/206 - 142s - 687ms/step - loss: 5.0426 - val_loss: 4.9733 - learning_rate: 0.0050 - lr: 0.0050
Epoch

In [ ]:
def BOTDA_plotfn(xvec, parmean, parstd, filename, mode='bgs',
                 figsize=(12,3), figfmt='svg', ylims=None):
    if mode == 'bgs':
        ylabels = ['BFS (MHz)', 'FWHM (MHz)']
        fig, ax = plt.subplots(1, 2, figsize=figsize)
        # Plot for BFS (first subplot)
        ax[0].plot(xvec, parmean[:, 0], 'k-', label='predicted mean')
        ax[0].set_xlabel('Fiber Length (km)')
        ax[0].set_ylabel(ylabels[0])
        ymin = parmean[:, 0] - 3 * parstd[:, 0]
        ymax = parmean[:, 0] + 3 * parstd[:, 0]
        ax[0].fill_between(xvec, ymin.flatten(), ymax.flatten(), color='blue', alpha=0.25, label='99% confidence interval')
        ax[0].legend(loc='lower right')
        ax[0].set_title(r"$\sigma = %5.3f$" % (np.std(parmean[:, 0])))
        ax[0].grid()
        if ylims is None:
            ax[0].set_ylim([0.98 * np.min(ymin), 1.02 * np.max(ymax)])
        else:
            ax[0].set_ylim(ylims[0])

        # Plot for FWHM (second subplot)
        ax[1].plot(xvec, parmean[:, 1], 'k-', label='predicted mean')
        ax[1].set_xlabel('Fiber Length (km)')
        ax[1].set_ylabel(ylabels[1])
        ymin = parmean[:, 1] - 3 * parstd[:, 1]
        ymax = parmean[:, 1] + 3 * parstd[:, 1]
        ax[1].fill_between(xvec, ymin.flatten(), ymax.flatten(), color='blue', alpha=0.25, label='99% confidence interval')
        ax[1].legend(loc='lower right')
        ax[1].set_title(r"$\sigma = %5.3f$" % (np.std(parmean[:, 1])))
        ax[1].grid()
        if ylims is None:
            ax[1].set_ylim([0.98 * np.min(ymin), 1.02 * np.max(ymax)])
        else:
            ax[1].set_ylim(ylims[1])

        plt.tight_layout(w_pad=2.0)

    elif mode == 'bps':
        ylabels = ['BFS (MHz)']
        fig, ax = plt.subplots(1, 1, figsize=figsize)
        ax.plot(xvec, parmean[:, 0], 'k-', label='predicted mean')
        ax.set_xlabel('Fiber Length (km)')
        ax.set_ylabel(ylabels[0])
        ymin = parmean[:, 0] - 3 * parstd[:, 0]
        ymax = parmean[:, 0] + 3 * parstd[:, 0]
        ax.fill_between(xvec, ymin.flatten(), ymax.flatten(), color='blue', alpha=0.25, label='99% confidence interval')
        ax.legend(loc='lower right')
        ax.set_title(r"$\sigma = %5.3f$" % (np.std(parmean[:, 0])))
        ax.grid()
        if ylims is None:
            ax.set_ylim([0.98 * np.min(ymin), 1.02 * np.max(ymax)])
        else:
            ax.set_ylim(ylims[0])

        plt.tight_layout(w_pad=2.0)

    plt.savefig(filename, format=figfmt, dpi=600)

def evaluate_botda_methods(raw_data, pdnn, wvec, x0, mode, xvec, plot_dir='.', figfmt='svg', ylims=None):
    """
    Function to evaluate curve fitting and PDNN methods for BOTDA data.

    Inputs:
    - raw_data: Input data for fitting and prediction.
    - pdnn: Deep learning model object with batch_predict method.
    - wvec: Frequency vector for least squares fitting.
    - x0: Initial guess for least squares fitting.
    - mode: 'bps' for BFS only, 'bgs' for BFS and FWHM.
    - xvec: Fiber length vector for plotting.
    - plot_dir: Directory to save plots.
    - figfmt: Format for saving figures.
    - ylims: Optional y-limits for plots.

    Outputs:
    - Returns a dictionary with computed variables, times, statistical evaluations, and comparisons.
    - Saves plots for LS, NN, and differences.
    """
    # Least squares curve fitting
    start = time.time()
    LS_parmat, LS_err = batch_ls_fit(raw_data, wvec, x0, mode=mode)
    curve_fitting_time = time.time() - start

    # PDNN prediction
    start = time.time()
    NN_par_mean, NN_par_std = pdnn.batch_predict(raw_data)
    PDNN_time = time.time() - start

    # Prepare results dictionary
    results = {
        'curve_fitting_time': curve_fitting_time,
        'PDNN_time': PDNN_time,
        'LS_par_mean': LS_parmat,
        'LS_par_std': LS_err,
        'NN_par_mean': NN_par_mean,
        'NN_par_std': NN_par_std
    }

    # Statistical evaluations and comparisons
    if mode == 'bps':
        bfs_ls = LS_parmat[:, 0]
        bfs_ls_std = LS_err[:, 0]
        bfs_nn = NN_par_mean[:, 0]
        bfs_nn_std = NN_par_std[:, 0]

        # Differences and errors for BFS
        diff_bfs = bfs_nn - bfs_ls
        mae_bfs = np.mean(np.abs(diff_bfs))
        rmse_bfs = np.sqrt(np.mean(diff_bfs**2))
        mean_diff_bfs = np.mean(diff_bfs)
        std_diff_bfs = np.std(diff_bfs)

        results.update({
            'mae_bfs': mae_bfs,
            'rmse_bfs': rmse_bfs,
            'mean_diff_bfs': mean_diff_bfs,
            'std_diff_bfs': std_diff_bfs
        })

        # Plots
        BOTDA_plotfn(xvec, LS_parmat, LS_err, f'{plot_dir}/ls_{mode}.{figfmt}', mode=mode, ylims=ylims)
        BOTDA_plotfn(xvec, NN_par_mean, NN_par_std, f'{plot_dir}/nn_{mode}.{figfmt}', mode=mode, ylims=ylims)

        # Difference plot for BFS
        fig, ax = plt.subplots(figsize=(12, 3))
        ax.plot(xvec, diff_bfs, 'r-', label='NN - LS')
        ax.set_xlabel('Fiber Length (km)')
        ax.set_ylabel('Difference BFS (MHz)')
        ax.set_title(f'BFS Difference (MAE: {mae_bfs:.3f}, RMSE: {rmse_bfs:.3f})')
        ax.legend()
        ax.grid()
        plt.tight_layout()
        plt.savefig(f'{plot_dir}/diff_bfs_{mode}.{figfmt}', format=figfmt, dpi=600)

    elif mode == 'bgs':
        bfs_ls = LS_parmat[:, 0]
        fwhm_ls = LS_parmat[:, 1]
        bfs_ls_std = LS_err[:, 0]
        fwhm_ls_std = LS_err[:, 1]
        bfs_nn = NN_par_mean[:, 0]
        fwhm_nn = NN_par_mean[:, 1]
        bfs_nn_std = NN_par_std[:, 0]
        fwhm_nn_std = NN_par_std[:, 1]

        # Differences and errors for BFS
        diff_bfs = bfs_nn - bfs_ls
        mae_bfs = np.mean(np.abs(diff_bfs))
        rmse_bfs = np.sqrt(np.mean(diff_bfs**2))
        mean_diff_bfs = np.mean(diff_bfs)
        std_diff_bfs = np.std(diff_bfs)

        # Differences and errors for FWHM
        diff_fwhm = fwhm_nn - fwhm_ls
        mae_fwhm = np.mean(np.abs(diff_fwhm))
        rmse_fwhm = np.sqrt(np.mean(diff_fwhm**2))
        mean_diff_fwhm = np.mean(diff_fwhm)
        std_diff_fwhm = np.std(diff_fwhm)

        results.update({
            'mae_bfs': mae_bfs,
            'rmse_bfs': rmse_bfs,
            'mean_diff_bfs': mean_diff_bfs,
            'std_diff_bfs': std_diff_bfs,
            'mae_fwhm': mae_fwhm,
            'rmse_fwhm': rmse_fwhm,
            'mean_diff_fwhm': mean_diff_fwhm,
            'std_diff_fwhm': std_diff_fwhm
        })

        # Plots
        BOTDA_plotfn(xvec, LS_parmat, LS_err, f'{plot_dir}/ls_{mode}.{figfmt}', mode=mode, ylims=ylims)
        BOTDA_plotfn(xvec, NN_par_mean, NN_par_std, f'{plot_dir}/nn_{mode}.{figfmt}', mode=mode, ylims=ylims)

        # Difference plot for BFS and FWHM
        fig, ax = plt.subplots(1, 2, figsize=(12, 3))
        ax[0].plot(xvec, diff_bfs, 'r-', label='NN - LS')
        ax[0].set_xlabel('Fiber Length (km)')
        ax[0].set_ylabel('Difference BFS (MHz)')
        ax[0].set_title(f'BFS Difference (MAE: {mae_bfs:.3f}, RMSE: {rmse_bfs:.3f})')
        ax[0].legend()
        ax[0].grid()

        ax[1].plot(xvec, diff_fwhm, 'r-', label='NN - LS')
        ax[1].set_xlabel('Fiber Length (km)')
        ax[1].set_ylabel('Difference FWHM (MHz)')
        ax[1].set_title(f'FWHM Difference (MAE: {mae_fwhm:.3f}, RMSE: {rmse_fwhm:.3f})')
        ax[1].legend()
        ax[1].grid()

        plt.tight_layout(w_pad=2.0)
        plt.savefig(f'{plot_dir}/diff_{mode}.{figfmt}', format=figfmt, dpi=600)

    return results

In [ ]:
bps_results = evaluate_botda_methods(real_bps, pdnn, wvec=wvec, x0=x0, mode='bps', xvec=xvec, plot_dir=RESULTS_DIR, figfmt='svg', ylims=None)

In [ ]:
pdnn_bgs = PDNN(wvec=frequency_axis_mhz, mode='bgs')
# pdnn_bgs.load_model()
pdnn_bgs.full_pipeline()

In [ ]:
bgs_results = evaluate_botda_methods(real_bgs, pdnn_bgs, wvec=wvec, x0=x0, mode='bgs', xvec=xvec, plot_dir=RESULTS_DIR, figfmt='svg', ylims=None)

In [ ]:
import pickle
res = {
    'bps':bps_results,
    'bgs':bgs_results
}
with open(f"{RESULTS_DIR}STATS.pickel", 'wb') as f:
  pickle.dump(res, f)